In [ ]:
# !pip install dotenv fyers_api
from fyers_api import fyersModel
from fyers_api import accessToken
import pandas as pd
import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv("../.env")
# print("running")

In [ ]:
client_id=os.environ.get('API_CLIENT_ID')
secret_key=os.environ.get('API_SECRET_KEY')
redirect_uri=os.environ.get('API_REDIRECT_URL')


In [ ]:
session=accessToken.SessionModel(client_id=client_id,
secret_key=secret_key,redirect_uri=redirect_uri, 
response_type='code', grant_type='authorization_code')

In [ ]:
auth_code_provider = session.generate_authcode()
print(auth_code_provider)

In [ ]:
auth_code=str(input("Enter the auth_code: "))

In [ ]:
session.set_token(auth_code)
response = session.generate_token()
access_token = response["access_token"]
if(access_token is None):
    print("Access token is not generated")
else:
    print("Access token is generated")
    

In [ ]:
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)
is_async = True

In [ ]:
fyers.get_profile()

In [ ]:
# Getting Data
def dynamicDataFetching(startDate,endDate,symbol):
    # Creating a dataframe to store the data
    df = pd.DataFrame(columns=["date","open","high","low","close","volume"])
    today = dt.date.today()
    range_from=startDate
    while(range_from<=today):
    # Check whether the upcoming endDate is lesser than today if we add 99D or else make it today
        if((range_from + dt.timedelta(days=99))<today):
            range_to = range_from + dt.timedelta(days=99)
        else:
            range_to = today

        # Api call 
        data = {"symbol":symbol,"resolution":"5","date_format":"1","range_from":range_from,"range_to":range_to,"cont_flag":"1"}
        res = fyers.history(data)
        res = res["candles"]
        
        # Converting the data into pandas dataframe 
        tempDf = pd.DataFrame(columns=["date","open","high","low","close","volume"])
        for i in range(len(res)):
             tempDf.loc[i]= [
                res[i][0],
                res[i][1],
                res[i][2],
                res[i][3],
                res[i][4],
                res[i][5]]
       
        # Appending the data to the main dataframe
        df = pd.concat([df,tempDf])

        # Updating the range_from to last date of last api call
        range_from=range_to+dt.timedelta(days=1)
    return df

In [ ]:
startDate = dt.date(2021,7,1)
endDate = dt.date.today()
symbol="NSE:SBIN-EQ"

df = dynamicDataFetching(startDate,endDate,symbol)
print(df.dtypes)
print(df.describe())

In [ ]:
print(df.head())

In [ ]:
# Converting everything to playable format + changing the time zone from utc to ist
df["date"]= pd.to_datetime(df["date"],unit='s',utc=True).dt.tz_convert('Asia/Kolkata')
df.set_index("date",inplace=True)
df["open"]=df["open"].astype(float)
df["high"]=df["high"].astype(float)
df["low"]=df["low"].astype(float)
df["close"]=df["close"].astype(float)
df["volume"]=df["volume"].astype(float)

print(df.head())
print("\nData types :\n" , df.dtypes)

In [ ]:
# Export df to csv
fileName = "SBI.csv"
# df.to_csv(f"../data/{fileName}")``